<a href="https://colab.research.google.com/github/mjsipes/stock_algorithms/blob/main/StockAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def perfectAlgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"][i]
        if i < len(data)-1:
            difference[i] = data["Close"][i] - data["Close"][i-1]
            pdifference[i] = (data["Close"][i] - data["Close"][i-1]) / data["Close"][i]
            direction[i] = "UP" if (data["Close"][i] - data["Close"][i-1]) > 0 else "DOWN"

            if i < len(data)-1 and data["Close"][i+1] > data["Close"][i]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"][i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif i < len(data)-1 and data["Close"][i+1] < data["Close"][i]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"][i] * (1 - fee_decimal)
                    total_profit = cash_from_sale-initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"][i]
    if DEBUG:
        print(f'{"Index":<10}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            print(f'{i:<10}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')


    return current_cash + stocks_owned * data["Close"][-1]


In [ ]:
def greedyalgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"][i]
        if i > 1:
            difference[i] = data["Close"][i] - data["Close"][i-1]
            pdifference[i] = (data["Close"][i] - data["Close"][i-1]) / data["Close"][i]
            direction[i] = "UP" if (data["Close"][i] - data["Close"][i-1]) > 0 else "DOWN"

            if data["Close"][i] > data["Close"][i - 1] and data["Close"][i-1] < data["Close"][i - 2]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"][i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif data["Close"][i] < data["Close"][i - 1] and data["Close"][i-1] > data["Close"][i - 2]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"][i] * (1 - fee_decimal)
                    total_profit = cash_from_sale-initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"][i]
    if DEBUG:
        print(f'{"Index":<10}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            print(f'{i:<10}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')


    return current_cash + stocks_owned * data["Close"][-1]



In [ ]:
import sys
import os
import yfinance as yf
ticker_symbol = "DOGE-USD"
initial_investment = 100
year = 2024
month = 5
day = 9
interval = "1m"
fee_percent = 0.1
DEBUG = True


original_stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
sys.stderr = original_stderr

# perfect_algo_profit = perfectAlgo(data, initial_investment,fee_percent, DEBUG)
greedy_algo_profit = greedyalgo(data, initial_investment,fee_percent, DEBUG)

Index     Price     Difference     %Difference    Direction Action    Networth  
0         0.14      0.00           0.00%          -         HOLD      0.00      
1         0.14      0.00           0.00%          -         HOLD      0.00      
2         0.14      0.00           0.16%          UP        HOLD      100.00    
3         0.14      0.00           0.04%          UP        HOLD      100.00    
4         0.14      0.00           0.09%          UP        HOLD      100.00    
5         0.14      0.00           0.01%          UP        HOLD      100.00    
6         0.14      -0.00          -0.00%         DOWN      HOLD      100.00    
7         0.14      0.00           0.01%          UP        BUY       99.90     
8         0.14      -0.00          -0.00%         DOWN      SELL      99.80     
9         0.14      -0.00          -0.00%         DOWN      HOLD      99.80     
10        0.14      0.00           0.04%          UP        BUY       99.70     
11        0.14      0.00    

In [ ]:
from datetime import datetime
def is_weekend(year, month, day):
    date_obj = datetime(year, month, day)
    return date_obj.weekday() >= 5

In [ ]:
import sys
import os
import yfinance as yf

def analyze_day(year, month, day, ticker_symbol, initial_investment, interval, fee_percent, DEBUG):
    original_stderr = sys.stderr
    sys.stderr = open(os.devnull, 'w')
    data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
    sys.stderr = original_stderr

    greedy_algo_profit = greedyalgo(data, initial_investment,fee_percent, DEBUG)
    perfect_algo_profit = perfectAlgo(data, initial_investment,fee_percent, DEBUG)

    return greedy_algo_profit, perfect_algo_profit


In [ ]:
def format_profit_list(profit_list):
    return ' '.join(f'{profit:8.2f}' for profit in profit_list)


In [ ]:
year = 2024
month = 4
day_begin = 12
day_end = 30
ticker_symbol_list = ["AAPL", "GOOG", "MSFT", "IBM", "NVDA", "RNG", "BTC-USD","ETH-USD", "DOGE-USD", "SHIB-USD", "ADA-USD"]
initial_investment = 100
interval = "5m"
fee_percent = 0.1 #binance maker take fee is 0.1%
DEBUG = False


for ticker_symbol in ticker_symbol_list:
    greedyprofit = []
    perfectprofit = []
    stockprice = []
    difference = []
    volatility = []
    for day in range(day_begin, day_end):
        if not is_weekend(year, month, day):
            cgreedyprofit, cperfectprofit = analyze_day(year, month, day, ticker_symbol, initial_investment,interval ,fee_percent, DEBUG)
            greedyprofit.append(cgreedyprofit)
            perfectprofit.append(cperfectprofit)


    # Calculate and print the month total profit
    average_daily_greedy_profit = sum(greedyprofit) / len(greedyprofit)
    average_daily_perfect_profit = sum(perfectprofit) / len(perfectprofit)
    print(f"{ticker_symbol} avg daily perfect profit: {average_daily_perfect_profit:.2f}")
    print(f"{ticker_symbol} avg daily greedy profit:  {average_daily_greedy_profit:.2f}")
    print(f"{ticker_symbol} daily perfect profit:     {format_profit_list(perfectprofit)}")
    print(f"{ticker_symbol} daily greedy profit:      {format_profit_list(greedyprofit)}")
    print()




AAPL avg daily perfect profit: 99.34
AAPL avg daily greedy profit:  95.59
AAPL daily perfect profit:       100.75   100.03   101.04    98.78    98.22    99.03    98.84    98.22   100.49    99.46    98.08    99.08
AAPL daily greedy profit:         96.27    94.84    95.48    95.50    95.49    97.05    95.66    94.81    96.59    95.59    95.23    94.58

GOOG avg daily perfect profit: 100.29
GOOG avg daily greedy profit:  96.28
GOOG daily perfect profit:        98.95    99.45   100.34   100.17    99.88   101.60   100.30    99.17    99.63   102.48   102.05    99.46
GOOG daily greedy profit:         94.60    95.25    96.50    94.92    97.05    98.79    97.43    95.58    96.51    97.57    96.54    94.58

MSFT avg daily perfect profit: 99.73
MSFT avg daily greedy profit:  96.46
MSFT daily perfect profit:        99.24    99.06   100.10    99.47    99.20    99.15   100.46    98.46    99.07   102.23   100.74    99.62
MSFT daily greedy profit:         96.30    95.97    96.89    95.98    95.52    9